# `keras`

Let's now introduce a python package called `keras` which will provide us with greater versatility in building neural networks than `sklearn`.

## What we will accomplish

In this notebook we will
- Introduce `keras`:
    - Discuss how to install it,
- Review its syntax and
- Demonstrate how to build feed forward networks in `keras`.

In [ ]:
## For data handling
import pandas as pd
import numpy as np

## For plotting
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("darkgrid")

Moving forward we will be building our neural networks with `keras`.

From their documentation:

<blockquote>
Keras is a deep learning API written in Python, running on top of the machine learning platform TensorFlow. It was developed with a focus on enabling fast experimentation. Being able to go from idea to result as fast as possible is key to doing good research.
</blockquote>

In this notebook we will lay out how to build feed forward multi-layer networks.

For reference, this material is being built by looking at the `keras` documentation <a href="https://keras.io/about/">https://keras.io/about/</a> and the book, <a href="https://github.com/letspython3x/Books/blob/master/Deep%20Learning%20with%20Python.pdf">Deep Learning with Python</a>. In particular this notebook's content comes from chapters 3 and 4 of that text.

## Installation

You may not have `keras` installed on your computer at this point. Try and run the following code chunk.

In [ ]:
import keras

# if the above did not work 
# uncomment and then try the below
# from tensorflow import keras

print(keras.__version__)

If this ran for you, then you already have `keras` installed. For reference, the version of `keras` that I was running while writing this notebook was `2.11.0`.

If you received an error while running that code chunk you likely need to install `keras` to move forward.

##### Using `pip`

If you use `pip` to install packages try running:

`pip install keras`

in your command line or terminal to install `keras`.

##### Using `conda`

If you use `conda` try what is recommended at this link, <a href="https://anaconda.org/conda-forge/keras">https://anaconda.org/conda-forge/keras</a>.

##### Apple M1 chip computers

If you have an Apple computer with an M1 chip you may need extra help. Earlier in 2022 the `keras` package installation instructions did not yet play nicely with such hardware. Try performing a web search for relevant instructions.

If you are unsure if you have an M1 chip, check with these instructions <a href="https://www.howtogeek.com/706226/how-to-check-if-your-mac-is-using-an-intel-or-apple-silicon-processor/">https://www.howtogeek.com/706226/how-to-check-if-your-mac-is-using-an-intel-or-apple-silicon-processor/</a>.

## Building a neural net with `keras`

### Classifying MNIST

We will mimic our `sklearn` networks from the last notebook and build an MNIST classifier, using the `keras` version of the data.

In [ ]:
## This imports datasets stored in keras
from keras.datasets import mnist

In [ ]:
## Here we load the data
(X_train, y_train),(X_test, y_test) = mnist.load_data()

print("og shape of X_train", np.shape(X_train))

##### Reshape the data to be a single column

We will now reshape the data so that it has $60{,}000$ observations ($10{,}000$ for the test set) of $28\times28$ pixels. This will give us a $60000 \times 784$ $2$-D `numpy` array ($10000 \times 784$ for the test set).

In [ ]:
X_train = X_train.reshape(-1,28*28)
X_test = X_test.reshape(-1,28*28)

print("The new shape of X_train is", np.shape(X_train))
print("The new shape of X_test is", np.shape(X_test))

X_train = X_train/255
X_test = X_test/255

### Building the networks

We start by importing all of the necessary pieces. We will explain each piece as we need it.

In [ ]:
## import things
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics
from keras.utils.np_utils import to_categorical

# for earlier versions of keras run this instead
# from keras.utils import to_categorical
# instead of running
# from keras.utils.np_utils import to_categorical
# or look up the documentation for your version


What we learned as feed forward networks in the last notebook are also called <i>dense</i> networks because they are fully connected graphs.

We will now walk through the process of making a dense neural networks using `keras`.

##### 1. Make an empty `model`

In [ ]:
## we first make an empty model
## Sequential means we'll make a group
## of a linear stack of layers
model = models.Sequential()

##### 2. Add the layers to the `model`

We will build the following neural network architecture in this step.

<img src="lecture_12_assets/mnist_net.png" width="70%"></img>

In [ ]:
###### ONLY RUN THIS ONCE! ##########

## you add a layer with .add()
## A Dense layer means a fully connected feedforward layer
## the 16 means the layer is 16 nodes tall
## activation='relu' means the layer uses a relu activation function
## the first layer needs to be told the shape of the input data
model

# we then add a second layer that is 16 nodes tall
# and uses the relu activation function
# note we don't need the input shape here, 
# it is inferred from the first layer's output
model


# finally we add the output layer
# this will have a single node, representing a probability
# that the observation has positive sentiment
# this is why the activation is a softmax
# for more information on the softmax see the Practice Problems
model

##### 3. Compile the model with an optimizer,  loss and metric

The optimizer we use is `rmsprop`, this is an algorithm implemented by `keras` to perform the backpropagation step in fitting the neural network.

The loss we use is `categorical_crossentropy` which stems from information theory, this is the "cost function" we discussed in notebook 3. This is a common and popular choice for classification problems. See the HW notebook for an explanation of crossentropy.

The metric we use is simply `accuracy`.

Note that there are more options for all three of these choices and if you are interested in seeing more check out the `keras` documentation, it has an excellent search bar. There is also the possibility to use your own custom inputs as well.

In [ ]:
## we compile the network like so
## call .compile
## set our optimizer, optimizer='rmsprop'
## set our loss, loss='categorical_crossentropy'
## set our desired metrics, metrics=['accuracy']
model.compile(optimizer='rmsprop', 
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

##### 4. Fit the model on the training data

We fit our model on the training data, we'll look at $100$ epochs that use batch gradient descent with a batch size of `512` observations per batch.

We first create a validation set, this will be additional input for the model.

In [ ]:
## First make the validation set
from sklearn.model_selection import train_test_split

X_train_train,X_val,y_train_train,y_val = train_test_split(X_train, y_train,
                                                          test_size=.2,
                                                          shuffle=True,
                                                          stratify=y_train,
                                                          random_state=440)

##### `to_categorical`

In order for the desired output of our model to play nicely with the training and validation data we have to first turn each observation into a categorical output with the `keras` function `to_categorical`. Let's see what this does.

In [ ]:
print(y_train)

print("\n\n\n")

print(to_categorical(y_train))

In [ ]:
## I now fit the model, and store the training history
## I use 100 epochs and a batch_size of 512
n_epochs = 
batch_size = 
history = model.fit()

##### 5. Examine epoch history loss and accuracy

The data stored in `history` includes a dictionary with epoch information on both the training and validation losses and accuracies.

In [ ]:
print(history_dict.keys())

In [ ]:
plt.figure(figsize = (8,6))

plt.scatter(range(1,n_epochs+1), history_dict['accuracy'], label = "Training Accuracy")
plt.scatter(range(1,n_epochs+1), history_dict['val_accuracy'], marker='v', label = "Validation Set Accuracy")

plt.xlabel("Epoch", fontsize=12)
plt.ylabel("Accuracy", fontsize=12)

plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.legend(fontsize=12)

plt.show()


plt.figure(figsize = (8,6))

plt.scatter(range(1,n_epochs+1), history_dict['loss'], label = "Training Loss")
plt.scatter(range(1,n_epochs+1), history_dict['val_loss'], marker='v', label = "Validation Set Loss")

plt.xlabel("Epoch", fontsize=12)
plt.ylabel("Loss Function Value", fontsize=12)

plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.legend(fontsize=12)

plt.show()

##### 6. Tuning the model architecture

Looking at data like this can allow us to choose a training period, i.e. the number of epochs, for a neural network as well as compare performance across two different networks. Let's try a comparison to a second network.

In [ ]:
model2 = models.Sequential()

model2.add(layers.Dense(32, activation='relu', input_shape=(28*28,)))
model2.add(layers.Dense(32, activation='relu'))
model2.add(layers.Dense(10, activation='softmax'))

model2.compile(optimizer = 'rmsprop',
                 loss = 'categorical_crossentropy',
                 metrics = ['accuracy'])

history2 = model2.fit(X_train_train,
                       to_categorical(y_train_train),
                       epochs = n_epochs,
                       batch_size = batch_size,
                       validation_data = (X_val, to_categorical(y_val)))

history_dict2 = history2.history

In [ ]:
plt.figure(figsize = (8,6))

plt.scatter(range(1,n_epochs+1), history_dict['val_accuracy'], label = "Neural Net 1 Val. Accuracy")
plt.scatter(range(1,n_epochs+1), history_dict2['val_accuracy'], marker='v', label = "Neural Net 2 Val. Accuracy")

plt.xlabel("Epoch", fontsize=12)
plt.ylabel("Accuracy", fontsize=12)

plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.legend(fontsize=12)

plt.show()

plt.figure(figsize = (8,6))

plt.scatter(range(1,n_epochs+1), history_dict['val_loss'], label = "Neural Net 1 Val. Loss")
plt.scatter(range(1,n_epochs+1), history_dict2['val_loss'], marker='v', label = "Neural Net 2 Val. Loss")

plt.xlabel("Epoch", fontsize=12)
plt.ylabel("Loss", fontsize=12)

plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.legend(fontsize=12)

plt.show()

##### 7. Selecting an architecture

It looks like the 32 x 32 network edges out the 16 x 16. Let's find the epoch that resulted in the lowest validation loss and use that for our final model.

In [ ]:
print("The epoch that had the lowest model 2 validation loss was",
     range(1,n_epochs)[np.argmin(history_dict2['val_loss'])])

In [ ]:
model2 = models.Sequential()

model2.add(layers.Dense(32, activation='relu', input_shape=(28*28,)))
model2.add(layers.Dense(32, activation='relu'))
model2.add(layers.Dense(10, activation='softmax'))

model2.compile(optimizer = 'rmsprop',
                 loss = 'categorical_crossentropy',
                 metrics = ['accuracy'])

history2 = model2.fit(X_train,
                       to_categorical(y_train),
                       epochs = range(1,n_epochs+1)[np.argmin(history_dict2['val_loss'])],
                       batch_size = 512,
                       validation_data = (X_val, to_categorical(y_val)))

history_dict2 = history2.history

#### 8. Predicting on the validation set

Let's now use this model to predict on the validation set. This is quite similar to the procedure you would follow in the `sklearn` setup.

In [ ]:
model2

In [ ]:
y_val[0]

Note that this produces a set of probabilities for each observation. For our prediction of an actual observation we can just choose the one with highest probability.

In [ ]:
np.argmax(model2.predict(X_val), axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
np.round(100*accuracy_score(y_val, np.argmax(model2.predict(X_val), axis=1)),2)

--------------------------

This notebook was written for the Erd&#337;s Institute C&#337;de Data Science Boot Camp by Matthew Osborne, Ph. D., 2023.

Any potential redistributors must seek and receive permission from Matthew Tyler Osborne, Ph.D. prior to redistribution. Redistribution of the material contained in this repository is conditional on acknowledgement of Matthew Tyler Osborne, Ph.D.'s original authorship and sponsorship of the Erdős Institute as subject to the license (see License.md)